In [1]:
import pulsar
import json
import matplotlib.pyplot as plt
import pandas as pd
from IPython.display import clear_output
import time

In [2]:
client = pulsar.Client('pulsar://localhost:6650')

In [3]:
producer = client.create_producer('request_scalability_w3')

2022-06-07 13:48:34.251 INFO  [140685324695360] ConnectionPool:85 | Created connection for pulsar://localhost:6650
2022-06-07 13:48:34.255 INFO  [140684370544384] ClientConnection:356 | [127.0.0.1:34092 -> 127.0.0.1:6650] Connected to broker
2022-06-07 13:48:34.260 INFO  [140684370544384] HandlerBase:54 | [persistent://public/default/request_scalability_w3, ] Getting connection from pool
2022-06-07 13:48:34.264 INFO  [140684370544384] ProducerImpl:170 | [persistent://public/default/request_scalability_w3, ] Created producer on broker [127.0.0.1:34092 -> 127.0.0.1:6650] 


In [4]:
total_data = 85400

In [5]:
test_stack = []

# Divides the request into batches and sends it to the analysis nodes
def makeRequest(data_size, batch_size=264):
    temp_total = data_size
    start_index = 0
    if not (0 < data_size < 365000):
        raise ValueError("Data Size not available!")
    while data_size > 0:
        if data_size - batch_size < 0:
            batch_size = data_size
        next_index = start_index
        next_index += batch_size
        producer.send((f"{start_index} - {next_index}").encode('utf-8'))
        #test_stack.append(f"{start_index} - {next_index}")
        start_index = next_index + 1
        data_size -= batch_size + 1

In [6]:
def aggregate_dicts(dict_a, dict_b):
    for key in dict_b.keys():
        if key not in dict_a.keys():
            dict_a[key] = int(dict_b[key])
        else:
            dict_a[key] += int(dict_b[key])
    return dict_a

def parse_dict(prog_dict):
    prog_df = pd.DataFrame.from_dict(prog_dict, orient='index', columns=["count"])
    prog_df = prog_df.sort_values("count", ascending=False)
    return prog_df

In [7]:
start_time = time.time()

In [8]:
consumer_one = client.subscribe('response_one', subscription_name='client')
consumer_two = client.subscribe('response_two', subscription_name='client')
consumer_three = client.subscribe('response_three', subscription_name='client')
consumer_four = client.subscribe('response_four', subscription_name='client')

2022-06-07 13:48:34.996 INFO  [140685324695360] Client:88 | Subscribing on Topic :response_one
2022-06-07 13:48:34.999 INFO  [140684370544384] HandlerBase:54 | [persistent://public/default/response_one, client, 0] Getting connection from pool
2022-06-07 13:48:35.003 INFO  [140684370544384] ConsumerImpl:216 | [persistent://public/default/response_one, client, 0] Created consumer on broker [127.0.0.1:34092 -> 127.0.0.1:6650] 
2022-06-07 13:48:35.003 INFO  [140685324695360] Client:88 | Subscribing on Topic :response_two
2022-06-07 13:48:35.005 INFO  [140684370544384] HandlerBase:54 | [persistent://public/default/response_two, client, 1] Getting connection from pool
2022-06-07 13:48:35.009 INFO  [140684370544384] ConsumerImpl:216 | [persistent://public/default/response_two, client, 1] Created consumer on broker [127.0.0.1:34092 -> 127.0.0.1:6650] 
2022-06-07 13:48:35.010 INFO  [140685324695360] Client:88 | Subscribing on Topic :response_three
2022-06-07 13:48:35.012 INFO  [140684370544384]

In [9]:
makeRequest(15000)

In [10]:
prog_dict = {}
test_prog_dict = {}
devop_prog_dict = {}
top_repos = []
total_num = 0
total_test = 0
total_devop_test = 0

In [ ]:
while True:
    # merge top ranking programming languages
    msg = consumer_one.receive()
    consumer_one.acknowledge(msg)
    prog_lang = json.loads(msg.data().decode("utf-8"))
    prog_dict = aggregate_dicts(prog_dict, prog_lang)
    
    # List top commits
    msg = consumer_two.receive()
    consumer_two.acknowledge(msg)
    top_repos.extend(json.loads(msg.data().decode("utf-8")))
    top_repos = sorted(top_repos, key=lambda tup: tup[1], reverse=True)
    top_repos = top_repos[:10]

    # Show total test
    msg = consumer_three.receive()
    consumer_three.acknowledge(msg)
    print(msg.data().decode("utf-8"))
    test_prog_lang = json.loads(msg.data().decode("utf-8"))
    test_prog_dict = aggregate_dicts(test_prog_dict, test_prog_lang)
    
    # Show total test + devops
    msg = consumer_four.receive()
    consumer_four.acknowledge(msg)
    devop_prog_lang = json.loads(msg.data().decode("utf-8"))
    devop_prog_dict = aggregate_dicts(devop_prog_dict, devop_prog_lang)
    
    clear_output(wait=True)
    
    print(top_repos[:10])
    prog_dict_df = parse_dict(prog_dict)
    print(prog_dict_df.iloc[:10])
    print(parse_dict(test_prog_dict).iloc[:10])
    print(parse_dict(devop_prog_dict).iloc[:10])
    print("run time: %s seconds" % (time.time() - start_time))

[['https://github.com/fabianishere/udm-kernel', 799619], ['https://github.com/leaningtech/cheerp-compiler', 423988], ['https://github.com/Status-Plus/StatusPlus', 396098], ['https://github.com/GiorgioComitini/COVID-19', 107359], ['https://github.com/vlggms/tegustation', 79729], ['https://github.com/abhijoshi2k/abhijoshi2k', 73278], ['https://github.com/SinguloStation13/SinguloStation13', 71417], ['https://github.com/feravolt/Brutal_busybox', 16916], ['https://github.com/hotio/duplicacy', 16383], ['https://github.com/chinnkarahoi/jd_scripts', 13429]]
                  count
Python             3031
JavaScript         2193
TypeScript         1123
C#                  801
Java                765
C++                 730
Jupyter Notebook    698
HTML                623
Go                  577
C                   491
                  count
Python              563
TypeScript          271
JavaScript          268
Go                  157
Dart                153
PHP                 141
C++         

In [ ]:
client.close()